<a href="https://colab.research.google.com/github/custom-hyper/CoinGeckoAPI/blob/main/CoinGecko_price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title Install Libraries { display-mode: "code" }
!pip install plotly
!pip install pytrends
!pip install pycoingecko
!pip install pandas
!pip install tweepy
!pip install requests
!pip install pycoingecko

  Created wheel for pytrends: filename=pytrends-4.8.0-py3-none-any.whl size=16126 sha256=af2eb714b035c5b60d58ca6ebafcb57afbd725327970ad71af5eb838a211cea4
  Stored in directory: /root/.cache/pip/wheels/07/6f/5c/8174f98dec1bfbc7d5da4092854afcbcff4b26c3d9b66b5183
Successfully built pytrends


In [3]:
#@title Available Coins { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

#General Information table

def coins_stats(url):
    """
    Generates a list of the coins on CoinGecko
    Args: url of the CoinGecko website

    """
    frames = []
    response = requests.get(url)
    data = response.json()

    
    token_list = pd.DataFrame(data)

    print('Stats: ')
    print(token_list.info(verbose=True))
    print(token_list.isnull().sum())
    print(token_list.describe())
   
    return token_list 


def main():
  url = "https://api.coingecko.com/api/v3/coins/list"

  coins_stats(url)
  

if __name__ == "__main__":
    main()

Stats: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13503 entries, 0 to 13502
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      13503 non-null  object
 1   symbol  13503 non-null  object
 2   name    13503 non-null  object
dtypes: object(3)
memory usage: 316.6+ KB
None
id        0
symbol    0
name      0
dtype: int64
            id symbol    name
count    13503  13503   13503
unique   13503  11224   13503
top     01coin   meta  01coin
freq         1     11       1


#  Price & Markets Data

CoinGecko website seems not to work for this query, check later

In [11]:
#@title  Bulk Download { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

 #suppressing carriers
id = token_list.id
currency = 'usd'
Horizon = 'max'


#loop  prices extraction to token list
try:

  start = 0
  ends =   100

  for i in id[start : ends]:

    response = requests.get("https://api.coingecko.com/api/v3/coins/%20{}/market_chart?vs_currency={}&days={}".format(i,currency,Horizon))
    output = response.text
    json_parsed = json.loads(output)
    
    
    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    full_table['id'] = i
    #full_table.loc(i)
    frames.append(full_table)

    print('downloading data for token: ' + str(i))

    time.sleep(60/50)
    #Status report
    
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop
df = single

print(single)

path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_csv( path + 'CoinGecko_price_markets.csv')




downloading data for token: 01coin
downloading data for token: 0-5x-long-algorand-token
downloading data for token: 0-5x-long-altcoin-index-token
downloading data for token: 0-5x-long-ascendex-token-token
downloading data for token: 0-5x-long-bitcoin-cash-token


KeyboardInterrupt: ignored

# Close Prices

In [ ]:
PS:#@title  Bulk Download { display-mode: "code" }
import requests
import datetime
import time
import json
from google.colab import data_table
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
import logging
import sys
from tqdm.notebook import tqdm

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

 #suppressing carriers
id = token_list.id
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list
try:

  start = 0
  ends =   100

  for i in id[start : ends]:

    response = requests.get("https://api.coingecko.com/api/v3/coins/{}/market_chart?vs_currency={}&days={}".format(i,currency,Horizon))
    output = response.text
    json_parsed = json.loads(output)
    

    #Convert data to list of dataframes
    full_table = pd.json_normalize(json_parsed)       
    full_table['id'] = i
    #full_table.loc(i)
    frames.append(full_table)
    print(full_table.head(1))
    #Slow down requestslo

    time.sleep(60/50)
    #Status report
    logging.info('Page '+ str(i) + ' – Rows ' + ' – source: '+ str(url))
except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop
df = single

print(single)
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_csv( path + 'CoinGecko_price_list.csv')




 # OHLC Prices 


Candlestick  body:

1 - 2 days: 30 minutes
3 - 30 days: 4 hours
31 and before: 4 days

In [ ]:
#@title Bulk Download { display-mode: "code" }

from google.colab import data_table
from pycoingecko import CoinGeckoAPI
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from __future__ import division
import requests
import datetime
import time
import json

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

cg = CoinGeckoAPI()
 #suppressing carriers
id = token_list.id
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list


start = 0
ends =   100
frames = []
try:
  for i in id[start : ends]:
  #Chart
    a = cg.get_coin_ohlc_by_id(id=i, vs_currency=currency, days = Horizon)

    df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])

    
    df['id'] = i
      #full_table.loc(i)
    frames.append(df)
    print(df.head( 1))
      #Slow down requestsloc

    time.sleep(60/50)
      #Status report

except ConnectionError:
    time.sleep(60)
 
single = pd.concat(frames) #Careful, this line must be outside of the for loop

    
path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

single.to_csv( path + 'CoinGecko_ohlc_timeseries.csv')


In [ ]:
#@title Generate  Charts { display-mode: "code" }

from google.colab import data_table
from pycoingecko import CoinGeckoAPI
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
from pytrends.request import TrendReq
from __future__ import division


import requests
import datetime
import time
import json

#Extract token list
url = "https://api.coingecko.com/api/v3/coins/list"
frames = []
response = requests.get(url)
data = response.json()
token_list = pd.DataFrame(data)

cg = CoinGeckoAPI()
 #suppressing carriers
id = token_list.id
currency = 'usd'
Horizon = 'max'

#loop  prices extraction to token list


start = 0
ends =   100

for i in id[start : ends]:
#Chart
    a = cg.get_coin_ohlc_by_id(id=i, vs_currency=currency, days = Horizon)

    df = pd.DataFrame(data=a, columns = ['Date','Open','High', 'Low', 'Close'])

    fig2 = go.Figure(data=[go.Candlestick(x=df['Date'],open=df['Open'], high=df['High'],low=df['Low'], close=df['Close'])])
                          
    fig2.show()
    


In [ ]:
import numpy as np
import pandas as pd

frames=[]

for rows in single['id']:
  for i in single['prices']:
      
    df = pd.DataFrame(i,columns = ['date','price'], index = None)
    df['id'] = rows

  frames.append(df)

combined = pd.concat(frames)

tokens = len(combined['id'].unique())


path = "/content/drive/MyDrive/workspace/pipelines/datasets/coingecko/coingecko-coin-list/output"

combined.to_csv(path+'/CoinGecko_prices.csv')

combined.info()

print('REPORT ')
print('Tokens total: '+ str(tokens))

In [ ]:
combined.head()

In [ ]:
combined.describe()